## Final Project Submission

Please fill out:
* Student name: Andy Martin del Campo
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Victor
* Blog post URL:


In [1]:
#get data
import pandas as pd

df = pd.read_csv('kc_house_data.csv')

In [33]:
#brief look at data
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [34]:
#get info on columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


### First Assumptions
1. It looks like the data all has the same number of entries. 

### Next Steps
-Go through and check each column for NaN values or any other interesting inputs


In [2]:
#check for NaN values
df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

- It appears that 'waterfront', 'view', and 'yr_renovated' have Nan values. 
- next step is to see if the Nan values are significant for each column and see if we can replace them with something meaningful


In [2]:
#check all values of 'waterfront'
print(df.waterfront.nunique())
print(df.waterfront.unique())
print(df.waterfront.value_counts())

2
[nan  0.  1.]
0.0    19075
1.0      146
Name: waterfront, dtype: int64


# What is the best approach here? Does NaN mean 0 or not on the waterfront? or is the data 'missing'. Is there a way to tell if it should be on the waterfront?


In [4]:
#look at the mean of 0, and 1 for waterfront. Then swithc NaN to 0 and test again. See if they are similar
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19221.000000,21534.000000,21597.000000,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007596,0.233863,3.409825,7.657915,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086825,0.765686,0.650546,1.173200,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [3]:
df_test = df.copy()
df_test.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
#set all NaN waterfront values to 0
def nan_replace(value):
    if value == 1:
        return value
    elif value == 0:
        return value
    else:
        return 0
    
df_test.waterfront = df_test.waterfront.map(lambda x: nan_replace(x))

In [5]:
df_test.isna().sum()
print(df_test.waterfront.nunique())
print(df_test.waterfront.unique())
print(df_test.waterfront.value_counts())

2
[0. 1.]
0.0    21451
1.0      146
Name: waterfront, dtype: int64


In [13]:
df_test.groupby('waterfront').describe()

id                                                         \
              count          mean           std          min           25%   
waterfront                                                                   
0.0         21451.0  4.581329e+09  2.876910e+09    1000102.0  2.123700e+09   
1.0           146.0  4.454969e+09  2.858045e+09  121029034.0  2.016369e+09   

                                                        price                \
                     50%           75%           max    count          mean   
waterfront                                                                    
0.0         3.904940e+09  7.307250e+09  9.900000e+09  21451.0  5.322862e+05   
1.0         3.692780e+09  7.390450e+09  9.808701e+09    146.0  1.717215e+06   

            ... sqft_living15         sqft_lot15                              \
            ...           75%     max      count          mean           std   
waterfront  ...                                                                
0.0         ...        2356.0  6210.0    21451.0  12689.299753  27287.285949   
1.0         ...        3140.0  4913.0      146.0  22893.698630  23264.085993   

                                                         
               min      25%      50%      75%       max  
waterfront                                               
0.0          651.0   5100.0   7601.0  10025.5  871200.0  
1.0         3230.0  11991.5  16260.5  24537.5  192268.0  

[2 rows x 144 columns]

In [14]:
df.groupby('waterfront').describe()

id                                                         \
              count          mean           std          min           25%   
waterfront                                                                   
0.0         19075.0  4.593352e+09  2.877189e+09    1000102.0  2.124079e+09   
1.0           146.0  4.454969e+09  2.858045e+09  121029034.0  2.016369e+09   

                                                        price                \
                     50%           75%           max    count          mean   
waterfront                                                                    
0.0         3.905081e+09  7.334401e+09  9.900000e+09  19075.0  5.326420e+05   
1.0         3.692780e+09  7.390450e+09  9.808701e+09    146.0  1.717215e+06   

            ... sqft_living15         sqft_lot15                              \
            ...           75%     max      count          mean           std   
waterfront  ...                                                                
0.0         ...        2350.0  6210.0    19075.0  12729.486186  27593.553043   
1.0         ...        3140.0  4913.0      146.0  22893.698630  23264.085993   

                                                         
               min      25%      50%      75%       max  
waterfront                                               
0.0          651.0   5100.0   7600.0  10031.5  871200.0  
1.0         3230.0  11991.5  16260.5  24537.5  192268.0  

[2 rows x 144 columns]

## looking at the values of the describe() function on the dataframe with NaN set to 0 and comparing it with the original values there was not a significant change so setting NaN to 0 is okay


### Next up is the view column